In [1]:
import pandas as pd
import geopandas as gpd

In [3]:
census_2016 = pd.read_csv('../data/SA2-P02_Selected_Medians_and_Averages-Census_2016.csv/sa2_p02_selected_medians_and_averages_census_2016-7489535763835097688.csv')
census_2016.columns = ['sa2_name_2016', 'Median_total_household_income_weekly', 
                       'Median_age_of_persons', 'Median_mortgage_repayment_monthly',
                       'Median_rent_weekly', 'Average_number_of_persons_per_bedroom',
                       'sa2_code_2016', 'Median_total_family_income_weekly',
                       'Average_household_size', 'Median_total_personal_income_weekly']
census_2016.drop('sa2_name_2016', axis=1, inplace=True)

In [6]:
GCCSA = gpd.read_file('../data/1270055001_gccsa_2016_aust_shape/GCCSA_2016_AUST.shp')
GCCSA = GCCSA.to_crs('EPSG:7855')
Greater_Melbourne = GCCSA.iloc[3]['geometry']

In [7]:
SA2 = gpd.read_file('../data/1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp')
SA2 = SA2.to_crs('EPSG:7855')

In [10]:
SA2_GreaterMelbourne = SA2[SA2.within(Greater_Melbourne)]
SA2_GreaterMelbourne = SA2_GreaterMelbourne[['SA2_MAIN16', 'SA2_NAME16', 'geometry']]
SA2_GreaterMelbourne['SA2_MAIN16'] = SA2_GreaterMelbourne['SA2_MAIN16'].astype('int64')

In [13]:
census_2016_geometry = pd.merge(census_2016, SA2_GreaterMelbourne, 
                               how='inner', left_on='sa2_code_2016', right_on='SA2_MAIN16').drop('sa2_code_2016', axis=1)

In [15]:
census_2016_geometry = gpd.GeoDataFrame(census_2016_geometry, geometry='geometry')
census_2016_geometry.to_csv('../output/preprocessing/census_SA2_GreaterMelbourne_2016.csv', index=False)